<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/rakuten_towercollector_unknown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# アップロード

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

# プログラム

In [ ]:
import pandas as pd

In [ ]:
df0 = pd.read_csv(fn, parse_dates=["measured_at"], dtype={"ta": str}).dropna(how="all", axis=1)

In [ ]:
df0.dtypes

In [ ]:
df0

In [ ]:
# 日時変換
df0["measured_at"] = df0["measured_at"].dt.tz_convert("Asia/Tokyo").dt.tz_localize(None)

In [ ]:
df0["ta"] = df0["ta"].fillna("-")

In [ ]:
df0.dropna(how="any", subset=["short_cell_id", "rnc"], inplace=True)

In [ ]:
# バンド指定
df1 = df0.query("mcc == 440 & mnc == 11").copy()
df1

In [ ]:
df1["short_cell_id"] = df1["short_cell_id"].astype(int)
df1["rnc"] = df1["rnc"].astype(int)

In [ ]:
df1["id"] = df1["short_cell_id"].astype(str) + "-" + df1["rnc"].astype(str)

In [ ]:
df2 = df1.groupby(["short_cell_id", "rnc"])["dbm"].max().reset_index()
df2

In [ ]:
df3 = pd.merge(df1, df2)

In [ ]:
df_pci = df3.drop_duplicates(subset=["short_cell_id", "rnc"]).loc[:, ["short_cell_id", "rnc", "psc"]]
df_pci.sort_values(["short_cell_id", "rnc"], inplace=True)
df_pci.to_csv("pci.csv", index=False)

In [ ]:
df3

# 地図

In [ ]:
import folium
from folium.features import DivIcon

In [ ]:
colors = [
    "black",
    "red",
    "green",
    "blue",
    "darkred",
    "darkgreen",
    "darkblue",
    "lightred",
    "lightgreen",
    "lightblue",
    "pink",
    "orange",
    "cadetblue",
    "beige",
    "gray",    
    "darkpurple",
    "lightgray",
    "white",
]

In [ ]:
def map_make(df):

    lat = df["lat"].mean()
    lon = df["lon"].mean()

    # map = folium.Map(location=[lat, lon], zoom_start=14)

    map = folium.Map(tiles = None, location=[lat, lon], zoom_start=14)

    # Googleマップ標準
    folium.raster_layers.TileLayer(
        'https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        subdomains = ['mt0','mt1','mt2','mt3'],
        name = "Google Map",
        attr = "<a href='https://developers.google.com/maps/documentation' target='_blank'>Google Map</a>"
    ).add_to(map)


    for i, r in df.iterrows():

        cid = r["rnc"] if r["rnc"] < 19 else 0

        folium.CircleMarker(
            location=[r.lat, r.lon],
            radius=10,
            color=colors[cid],
            fill=True,
            tooltip=r["id"],
        ).add_to(map)

        folium.Marker(
            location=[r.lat, r.lon],
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(10, 10),
                html=f'<div style="text-align:center; font-size: 12pt; font-weight: bold">{r["ta"]}</div>',
            ),
            tooltip=f'{r["id"]} TA:{r["ta"]}',
        ).add_to(map)

    return map

In [ ]:
df1

# 全部

In [ ]:
map0 = map_make(df1)
map0

In [ ]:
map0.save("map0.html")

# 最良値

In [ ]:
map1 = map_make(df3)
map1

In [ ]:
map1.save("map1.html")

# 未発見

In [ ]:
# スプレッドシートのURL
csv_url = "https://raku10ehime.github.io/map/ehime.csv"

In [ ]:
df4 = pd.read_csv(csv_url, index_col=0).dropna(subset=["eNB-LCID"])
df4

In [ ]:
df4["eNB-LCID"] = df4["eNB-LCID"].str.split("\n")
df5 = df4.explode("eNB-LCID")

In [ ]:
df5[["eNB", "LCID"]] = df5["eNB-LCID"].str.split("-", expand=True)

In [ ]:
df5["LCID"] = df5["LCID"].str.split(",")

In [ ]:
df6 = df5.explode("LCID")
df6["id"] = df6["eNB"] + "-" + df6["LCID"]
df6

In [ ]:
df7 = df3[~df3["id"].isin(df6["id"])]
df7

In [ ]:
print(df7.reindex(columns=["lat", "lon", "ta"]).to_csv(index=None, header=None))

## PCI

In [ ]:
print(df7.sort_values(by="ta").drop_duplicates(subset="id").sort_values(by="cell_id").reindex(columns=["id", "psc"]).to_csv(index=None, header=None))

In [ ]:
df7.to_csv("unknown.tsv", sep="\t")

In [ ]:
if len(df7) > 0:
    map2 = map_make(df7)

else:
    print("データなし")

In [ ]:
map2

In [ ]:
map2.save("map2.html")